In [7]:
using DynamicPolynomials, SparseArrays, LinearAlgebra

println("***Problem setting***")

n=20

println("Number of variable: n=",n)
println("====================")

@ncpolyvar x[1:n]# variables


function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end
function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,2))
v+=star_algebra.(v)
v=v./2
f=generate_random_poly(v)


# unit sphere constraint
m=n
q=floor(Int64,n/m)
R=ones(Float64,m)./n
T=[(j-1)*q+1:j*q for j in 1:m-1]
append!(T,[(m-1)*q+1:n])

g=[R[j]-sum(x[T[j]].^2) for j in 1:m]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")


l=ceil(Int64, n/7)


h=Vector{Polynomial{false,Float64}}(undef,l)
randx=[2*rand(length(T[j])).-1 for j in 1:m]# create a feasible solution
randx=[sqrt(R[j])*rand(1)[1]*randx[j]/norm(randx[j]) for j in 1:m]
randx=vcat(randx...)

for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=20
Number of inequality constraints: m=20
Number of equality constraints: l=3


In [8]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(x,f,g,h);

In [9]:
k=2
println("Relaxed order: k=",k)
println("====================")
t=1
println("Sparse order: t=",t)

Relaxed order: k=2
Sparse order: t=1


In [4]:
using NCTSSOS


@time begin
j=1
opt,data=nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),
        [[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,reducebasis=false,obj="eigen",
        merge=false,TS="block",QUIET=false)
while j <= t-1
    opt,data=nctssos_higher!(NCTSSOS.cdata_type(data),TS="block")
end
end

------------------------------------------------------
The sizes of blocks:
[26, 5]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 341             
  Cones                  : 0               
  Scalar variables       : 22              
  Matrix variables       : 7               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    

In [10]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val=ctpNCPOP.POP_TS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0000000000000013


  Number of blocks: omega=42
  Size of the largest block: s^max=401
  Number of equality trace constraints: zeta=5254
Modeling time:
  3.534610 seconds (12.38 M allocations: 3.876 GiB, 23.67% gc time)
iter=1   val=-11.500709619552952   gap=2.8420096033936613e-16   feas=2.613216694219691
iter=2   val=-3.8335698731843175   gap=0.7494694960948021   feas=1.0185275257235127
iter=4   val=-3.688110518382438   gap=0.6333258490910048   feas=0.642145956845726
iter=8   val=-3.2684848409784015   gap=0.5559747013275055   feas=0.4588661836120069
iter=16   val=-4.062161696001279   gap=0.0612192363501792   feas=0.32274041433630546
iter=32   val=-3.729718522254541   gap=0.03237461046644143   feas=0.18205097749169044
iter=64   val=-3.14952220703479   gap=0.09520009083844683   feas=0.08972229744562936
iter=128   val=-2.814017360754863   gap=0.2121596599211144   feas=0.03460964282406259
iter=256   val=-2.7241167142394564   gap=0.16432151887385743   feas=0.01793678429305976
iter=512   val=-2.73350742063639

-2.7551749247972914